In [1]:
# main imports to run keras model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#import dataset using pandas
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:, 13].values
print(X)
print(y)

[[619 'France' 'Female' ..., 1 1 101348.88]
 [608 'Spain' 'Female' ..., 0 1 112542.58]
 [502 'France' 'Female' ..., 1 0 113931.57]
 ..., 
 [709 'France' 'Female' ..., 0 1 42085.58]
 [772 'Germany' 'Male' ..., 1 0 92888.52]
 [792 'France' 'Female' ..., 1 0 38190.78]]
[1 0 1 ..., 1 1 0]


In [2]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:,1]=labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:,2]=labelencoder_X_2.fit_transform(X[:,2])
print(X)
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()
print(X)
X=X[:,1:]
print(X)

[[619 0 0 ..., 1 1 101348.88]
 [608 2 0 ..., 0 1 112542.58]
 [502 0 0 ..., 1 0 113931.57]
 ..., 
 [709 0 0 ..., 0 1 42085.58]
 [772 1 1 ..., 1 0 92888.52]
 [792 0 0 ..., 1 0 38190.78]]
[[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.01348880e+05]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    1.00000000e+00   1.12542580e+05]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    0.00000000e+00   1.13931570e+05]
 ..., 
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    1.00000000e+00   4.20855800e+04]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    0.00000000e+00   9.28885200e+04]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    0.00000000e+00   3.81907800e+04]]
[[  0.00000000e+00   0.00000000e+00   6.19000000e+02 ...,   1.00000000e+00
    1.00000000e+00   1.01348880e+05]
 [  0.00000000e+00   1.

In [3]:
#splitting the dataset into Training Set and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

#Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [6]:
#import from keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
#Initialize ANN
classifier = Sequential()
# Adding inputlayer and hiddenlayer with dropout
classifier.add(Dense(activation='relu', input_dim=11, units=6, kernel_initializer='uniform' ))
classifier.add(Dropout(p=0.1))
#Hidden layer
classifier.add(Dense(activation='relu', units=6, kernel_initializer='uniform'))
classifier.add(Dropout(p=0.1))
#output layer
classifier.add(Dense(activation='sigmoid', units=1, kernel_initializer='uniform'))

#Compile ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Fitting ANN to the training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

In [ ]:
#predicting test set results
y_pred = classifier.predict(X_test)
y_pred =(y_pred > 0.5)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
print ((1550+136)/2000)

In [ ]:
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

print("new prediction:", new_prediction)

In [7]:
#Evaluating, improving and Tuning the ANN

# Evaluating
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    #Initialize ANN
    classifier = Sequential()
    # Adding inputlayer and hiddenlayer
    classifier.add(Dense(activation='relu', input_dim=11, units=6, kernel_initializer='uniform' ))
    #Hidden layer
    classifier.add(Dense(activation='relu', units=6, kernel_initializer='uniform'))
    #output layer
    classifier.add(Dense(activation='sigmoid', units=1, kernel_initializer='uniform'))
    #Compile ANN
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

#classifier = KerasClassifier(build_fn=build_classifier, batch_size = 10, epochs=100)
classifier = KerasClassifier(build_fn=build_classifier)
#accuracies = cross_val_score(estimator = classifier, X=X_train, y=y_train, cv = 10, n_jobs = -1)
parameters = {'batch_size':[25,32], 'epochs':[100, 500], 'optimizer':['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', cv=10)
grid_search = grid_search.fit(X_train, y_train)

print("best_params:", grid_search.best_params_)
print("best_accuracy:", grid_search.best_score_)

Epoch 1/100
7200/7200 [==============================] - 1s 136us/step - loss: 0.5547 - acc: 0.7967
Epoch 2/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4366 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 1s 98us/step - loss: 0.4291 - acc: 0.7971
Epoch 4/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4254 - acc: 0.7971
Epoch 5/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4218 - acc: 0.7971
Epoch 6/100
7200/7200 [==============================] - 1s 96us/step - loss: 0.4192 - acc: 0.7996
Epoch 7/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4171 - acc: 0.8204
Epoch 8/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4161 - acc: 0.8249
Epoch 9/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4151 - acc: 0.8275
Epoch 10/100
7200/7200 [==============================] - 1s 105us/step - loss: 0.4141 - acc: 0.8308
Epoch 

7200/7200 [==============================] - 1s 117us/step - loss: 0.4009 - acc: 0.8337
Epoch 83/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4008 - acc: 0.8349
Epoch 84/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4005 - acc: 0.8336
Epoch 85/100
7200/7200 [==============================] - 1s 112us/step - loss: 0.4009 - acc: 0.8343
Epoch 86/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4005 - acc: 0.8349
Epoch 87/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4004 - acc: 0.8344
Epoch 88/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.4002 - acc: 0.8349
Epoch 89/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4009 - acc: 0.8351
Epoch 90/100
7200/7200 [==============================] - 1s 106us/step - loss: 0.4006 - acc: 0.8354
Epoch 91/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4005 - acc: 0.8371
Epoch

7200/7200 [==============================] - 1s 95us/step - loss: 0.4004 - acc: 0.8353
Epoch 63/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4003 - acc: 0.8358
Epoch 64/100
7200/7200 [==============================] - 1s 98us/step - loss: 0.4002 - acc: 0.8342
Epoch 65/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.4001 - acc: 0.8360
Epoch 66/100
7200/7200 [==============================] - 1s 130us/step - loss: 0.4000 - acc: 0.8356
Epoch 67/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4003 - acc: 0.8354
Epoch 68/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.3999 - acc: 0.8365
Epoch 69/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4000 - acc: 0.8350
Epoch 70/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4000 - acc: 0.8353
Epoch 71/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4000 - acc: 0.8361
Epoch 72/1

7200/7200 [==============================] - 1s 106us/step - loss: 0.4032 - acc: 0.8365
Epoch 43/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4031 - acc: 0.8351
Epoch 44/100
7200/7200 [==============================] - 1s 98us/step - loss: 0.4027 - acc: 0.8357
Epoch 45/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4028 - acc: 0.8356
Epoch 46/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4025 - acc: 0.8354
Epoch 47/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4028 - acc: 0.8337
Epoch 48/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4025 - acc: 0.8349
Epoch 49/100
7200/7200 [==============================] - 1s 102us/step - loss: 0.4024 - acc: 0.8358
Epoch 50/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.4023 - acc: 0.8347
Epoch 51/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4027 - acc: 0.8340
Epoch 5

7200/7200 [==============================] - 1s 97us/step - loss: 0.4045 - acc: 0.8347
Epoch 23/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4041 - acc: 0.8350
Epoch 24/100
7200/7200 [==============================] - 1s 91us/step - loss: 0.4034 - acc: 0.8362
Epoch 25/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4029 - acc: 0.8350
Epoch 26/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4027 - acc: 0.8342
Epoch 27/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4021 - acc: 0.8360
Epoch 28/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4020 - acc: 0.8351
Epoch 29/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4022 - acc: 0.8339
Epoch 30/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.4014 - acc: 0.8364
Epoch 31/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.4011 - acc: 0.8360
Epoch 32/100

...

8000/8000 [==============================] - 1s 102us/step - loss: 0.3990 - acc: 0.8364
Epoch 151/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3992 - acc: 0.8349
Epoch 152/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.3991 - acc: 0.8357
Epoch 153/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3989 - acc: 0.8337
Epoch 154/500
8000/8000 [==============================] - 1s 98us/step - loss: 0.3988 - acc: 0.8359
Epoch 155/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3989 - acc: 0.8359
Epoch 156/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3990 - acc: 0.8351
Epoch 157/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3990 - acc: 0.8353
Epoch 158/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3991 - acc: 0.8348
Epoch 159/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3992 - acc: 0.8344
Ep

8000/8000 [==============================] - 1s 106us/step - loss: 0.3986 - acc: 0.8359
Epoch 231/500
8000/8000 [==============================] - 1s 106us/step - loss: 0.3989 - acc: 0.8360
Epoch 232/500
8000/8000 [==============================] - 1s 107us/step - loss: 0.3987 - acc: 0.8345
Epoch 233/500
8000/8000 [==============================] - 1s 103us/step - loss: 0.3990 - acc: 0.8354
Epoch 234/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3989 - acc: 0.8354
Epoch 235/500
8000/8000 [==============================] - 1s 104us/step - loss: 0.3989 - acc: 0.8356
Epoch 236/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3989 - acc: 0.8341
Epoch 237/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3989 - acc: 0.8350
Epoch 238/500
8000/8000 [==============================] - 1s 102us/step - loss: 0.3986 - acc: 0.8345
Epoch 239/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3988 - acc: 0.835

8000/8000 [==============================] - 1s 99us/step - loss: 0.3987 - acc: 0.8356
Epoch 311/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3988 - acc: 0.8359
Epoch 312/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3986 - acc: 0.8360
Epoch 313/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3988 - acc: 0.8354
Epoch 314/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3984 - acc: 0.8349
Epoch 315/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3987 - acc: 0.8355
Epoch 316/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3988 - acc: 0.8360
Epoch 317/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3983 - acc: 0.8359
Epoch 318/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3990 - acc: 0.8364
Epoch 319/500
8000/8000 [==============================] - 1s 111us/step - loss: 0.3988 - acc: 0.8351
Epo

8000/8000 [==============================] - 1s 94us/step - loss: 0.3982 - acc: 0.8365
Epoch 391/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3985 - acc: 0.8353
Epoch 392/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3984 - acc: 0.8356
Epoch 393/500
8000/8000 [==============================] - 1s 104us/step - loss: 0.3985 - acc: 0.8350
Epoch 394/500
8000/8000 [==============================] - 1s 111us/step - loss: 0.3984 - acc: 0.8353
Epoch 395/500
8000/8000 [==============================] - 1s 106us/step - loss: 0.3985 - acc: 0.8360
Epoch 396/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3983 - acc: 0.8345
Epoch 397/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3984 - acc: 0.8359
Epoch 398/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3984 - acc: 0.8350
Epoch 399/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3984 - acc: 0.8355
E

8000/8000 [==============================] - 1s 105us/step - loss: 0.3983 - acc: 0.8355
Epoch 471/500
8000/8000 [==============================] - 1s 106us/step - loss: 0.3980 - acc: 0.8349
Epoch 472/500
8000/8000 [==============================] - 1s 109us/step - loss: 0.3982 - acc: 0.8349
Epoch 473/500
8000/8000 [==============================] - 1s 102us/step - loss: 0.3979 - acc: 0.8365
Epoch 474/500
8000/8000 [==============================] - 1s 105us/step - loss: 0.3981 - acc: 0.8355
Epoch 475/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3982 - acc: 0.8355
Epoch 476/500
8000/8000 [==============================] - 1s 100us/step - loss: 0.3979 - acc: 0.8357
Epoch 477/500
8000/8000 [==============================] - 1s 107us/step - loss: 0.3979 - acc: 0.8357
Epoch 478/500
8000/8000 [==============================] - 1s 109us/step - loss: 0.3983 - acc: 0.8341
Epoch 479/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.3977 - acc: 0.